In [ ]:
import json
import os
import pandas as pd
from matplotlib import pyplot as plt

data_dir = "./data"
if not os.path.exists(data_dir):
  os.makedirs(data_dir)

res_dir = "./results"
if not os.path.exists(res_dir):
  os.makedirs(res_dir)

speech_df = pd.read_csv(os.path.join(data_dir, "test.csv"))

messages = speech_df["Message"].tolist()
labels = speech_df["Aggressive"].tolist()

with open(os.path.join(res_dir, "pre_trained_pred.json"), "r") as f:
  pre_trained_pred = json.load(f)
with open(os.path.join(res_dir, "fine_tuned_pred.json"), "r") as f:
  fine_tuned_pred = json.load(f)
with open(os.path.join(res_dir, "perspective_indices.json"), "r") as f:
  perspective_indices = json.load(f)
with open(os.path.join(res_dir, "perspective_probs.json"), "r") as f:
  perspective_probs = json.load(f)
with open(os.path.join(res_dir, "perspective_pred.json"), "r") as f:
  perspective_pred = json.load(f)

print(len(pre_trained_pred), len(fine_tuned_pred), len(perspective_pred))

labels = [label for i, label in enumerate(labels) if i in perspective_indices]
pre_trained_pred = [pred for i, pred in enumerate(pre_trained_pred) if  i in perspective_indices]
fine_tuned_pred = [pred for i, pred in enumerate(fine_tuned_pred) if  i in perspective_indices]

print(len(pre_trained_pred), len(fine_tuned_pred), len(perspective_pred), len(labels))

def get_classification_result(y_true, y_pred):
  result = {"TP": [], "FP": [], "FN": [], "TN": []}
  for i, (true, pred) in enumerate(zip(y_true, y_pred)):
    if true == 1 and pred == 1:
      result["TP"].append(i)
    elif true == 1 and pred == 0:
      result["FN"].append(i)
    elif true == 0 and pred == 1:
      result["FP"].append(i)
    elif true == 0 and pred == 0:
      result["TN"].append(i)
  return result

def get_classification_metrics(classification_result):
  metrics = {}

  true_pos = len(classification_result["TP"])
  false_pos = len(classification_result["FP"])
  false_neg = len(classification_result["FN"])
  true_neg = len(classification_result["TN"])

  metrics["tp"] = true_pos
  metrics["fp"] = false_pos
  metrics["fn"] = false_neg
  metrics["tn"] = true_neg
  metrics["accuracy"] = (true_pos + true_neg) / (true_pos + false_pos + false_neg + true_neg)
  metrics["recall"] = true_pos / (true_pos + false_neg)
  metrics["precision"] = true_pos / (true_pos + false_pos)

  return metrics

pretrain_result = get_classification_result(labels, pre_trained_pred)
fine_tuned_result = get_classification_result(labels, fine_tuned_pred)
perspective_result = get_classification_result(labels, perspective_pred)

pretrain_metrics = get_classification_metrics(pretrain_result)
fine_tuned_metrics = get_classification_metrics(fine_tuned_result)
perspective_metrics = get_classification_metrics(perspective_result)

print("Pre-trained Model")
print(pretrain_metrics)

print("Fine-tuned Model")
print(fine_tuned_metrics)

print("Perspective API")
print(perspective_metrics)

6421 6421 6370
6370 6370 6370 6370
{'tp': 1015, 'fp': 2929, 'fn': 692, 'tn': 1734, 'accuracy': 0.43155416012558867, 'recall': 0.5946104276508495, 'precision': 0.25735294117647056}
{'tp': 1398, 'fp': 330, 'fn': 309, 'tn': 4333, 'accuracy': 0.8996860282574568, 'recall': 0.8189806678383128, 'precision': 0.8090277777777778}
{'tp': 969, 'fp': 169, 'fn': 738, 'tn': 4494, 'accuracy': 0.8576138147566719, 'recall': 0.5676625659050967, 'precision': 0.851493848857645}
